In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds

In [ ]:
from tensorflow.keras.layers import Normalization, Dense, InputLayer,Conv2D, MaxPool2D, Flatten, BatchNormalization,Input,Layer
from tensorflow.keras.losses import MeanSquaredError, Huber, MeanAbsoluteError, BinaryCrossentropy
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, shuffle_files=True, split=['train'])

In [ ]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  dataset_size = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * dataset_size))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO * dataset_size))
  val_dataset = val_test_dataset.take(int(VAL_RATIO * dataset_size))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO * dataset_size))
  return train_dataset, val_dataset, test_dataset

def resize_rescale(image, label):
  image = tf.image.resize(image, (224,224))/255.0
  return image, label

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset , val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size=8,
                                    reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.shuffle(buffer_size=8,
                                    reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

SEQUENTIAL API

In [ ]:
lenet_model = tf.keras.Sequential([
    InputLayer(input_shape = (224,224,3)),
    tf.keras.layers.Conv2D(filters = 6, kernel_size=3, strides=1, padding="valid",activation ='relu'),
    BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),

    tf.keras.layers.Conv2D(filters = 16, kernel_size=3, strides=1, padding="valid",activation ='relu'),
    BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),

    Flatten(),

    Dense(100, activation ="relu"),
    BatchNormalization(),
    Dense(10, activation ="relu"),
    BatchNormalization(),
    Dense(1, activation ="sigmoid")])
lenet_model.summary()

In [ ]:
lenet_model.compile(optimizer = Adam(learning_rate=0.01),
            loss = BinaryCrossentropy(),
            metrics= ['accuracy']
)

In [ ]:
history = lenet_model.fit(train_dataset, validation_data = val_dataset, epochs = 5, verbose = 1)

test_dataset= test_dataset.batch(1)

lenet_model.evaluate(test_dataset)


In [ ]:
lenet_model.save('C:/Users/Admin/Desktop/Tensor/lenet.h5')

In [ ]:
abc = tf.keras.models.load_model('lenet.h5')
abc.summary()

FUNCTIONAL API

In [ ]:
func_input = Input(shape = (224,224,3), name='input image')

x = tf.keras.layers.Conv2D(filters = 6, kernel_size=3, strides=1, padding="valid",activation ='relu')(func_input)
x = BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=2,strides=2)(x)

x = tf.keras.layers.Conv2D(filters = 16, kernel_size=3, strides=1, padding="valid",activation ='relu')(x)
x = BatchNormalization()(x)
output = tf.keras.layers.MaxPool2D(pool_size=2,strides=2)(x)


In [ ]:
feature_extract_model = Model(func_input, output,name='Feature_Extractor')
feature_extract_model.summary()

In [ ]:
func_input = Input(shape = (224,224,3), name='input image')

x = feature_extract_model(func_input)

x = Flatten()(x)

x = Dense(100, activation ="relu")(x)
x = BatchNormalization()(x)
x = Dense(10, activation ="relu")(x)
x = BatchNormalization()(x)
func_output = Dense(1, activation ="sigmoid")(x)

lenet_model1 = Model(func_input, func_output, name= 'Lenet_Model')
lenet_model1.summary()

In [ ]:
lenet_model1.compile(optimizer = Adam(learning_rate=0.01),
            loss = BinaryCrossentropy(),
            metrics= ['accuracy']
)
history = lenet_model1.fit(train_dataset, validation_data = val_dataset, epochs = 5, verbose = 1)

MODEL SUBCLASSING

In [ ]:
class FeatureExtractor(Layer):
    def __init__(self, filters, kernel_size, strides, padding, activation, pool_size,):
        super(FeatureExtractor, self).__init__()

        self.conv_1 = Conv2D(filters = 8, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu')
        self.batch_1 = BatchNormalization()
        self.pool_1 = MaxPool2D (pool_size = 2, strides= 2)

        self.conv_2 = Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu')
        self.batch_2 = BatchNormalization()
        self.pool_2 = MaxPool2D (pool_size = 2, strides= 2)
    def call(self, x):
        x = self.conv_1(x)
        x = self.batch_1(x)
        x = self.pool_1(x)

        x = self.conv_2(x)
        x = self.batch_2(x)
        x = self.pool_2(x)

        return x
    
func_input = tf.keras.Input(shape = (224,224,3), name='input image')

feature_sub_classed = FeatureExtractor(8,3,1,"valid","relu",2)

x = feature_sub_classed(func_input)

x = Flatten()(x)

x = Dense(100, activation ="relu")(x)
x = BatchNormalization()(x)
x = Dense(10, activation ="relu")(x)
x = BatchNormalization()(x)
func_output = Dense(1, activation ="sigmoid")(x)

lenet_subclass_model = Model(func_input, func_output, name= 'Lenet_Model2')
lenet_subclass_model.summary()

In [ ]:
lenet_subclass_model.compile(optimizer = Adam(learning_rate=0.01),
            loss = BinaryCrossentropy(),
            metrics= ['accuracy']
)
history = lenet_subclass_model.fit(train_dataset, validation_data = val_dataset, epochs = 5, verbose = 1)

In [ ]:
class LenetModel(Model):
    def __init__(self):
        super(LenetModel,self).__init__()
        self.feature_extractor = FeatureExtractor(8, 3, 1, "valid", "relu", 2)

        self.flatten = Flatten()

        self.dense_1 = Dense(100, activation = "relu")
        self.batch_1 = BatchNormalization()

        self.dense_2 = Dense(10, activation = "relu")
        self.batch_2 = BatchNormalization()

        self.dense_3 = Dense(1, activation = "sigmoid")

    def call(self, x):
        x = self.feature_extractor(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.batch_1(x)
        x = self.dense_2(x)
        x = self.batch_2(x)
        x = self.dense_3(x)
       
        return x
lenet_sub_classed = LenetModel()
lenet_sub_classed(tf.ones([1,224,224,3]))
lenet_sub_classed.summary()


In [ ]:
lenet_sub_classed.compile(optimizer = Adam(learning_rate=0.01),
            loss = BinaryCrossentropy(),
            metrics= ['accuracy']
)

In [ ]:
lenet_sub_classed.fit(train_dataset, validation_data = val_dataset, epochs = 5, verbose = 1)